# Pre-Requisites
## Install [Ollama](https://github.com/ollama/ollama)

```cmd
curl -fsSL https://ollama.com/install.sh | sh
```

## Run Ollama

```cmd
ollama start
```

## Run New Terminal and run model

```cmd
ollama run phi3
```

## Run Jupyter Notebook

```cmd
pipenv shell
pipenv run jupyter notebook
```

# Running Ollama and Phi3 Model Locally

In [1]:
import json

with open('../01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dic in docs_raw:
    for doc in course_dic['documents']:
        doc['course'] = course_dic['course']
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
import minsearch
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [4]:
import os
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [5]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

def llm(prompt, generate_params=None):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [6]:
llm('write that this is a test')

' This is a test.\n\n\nThe instruction seems to be simple, asking for the writing of just one sentence with specific content and purpose in mind – confirming whether an environment or system works correctly by executing what appears to be straightforward commands. The simplicity lies both in its directness (a command that should yield minimal output if functional) as well as a single-step execution without additional complexity like loops, conditional statements, external resources, or advanced coding techniques commonly included at higher difficulty levels.'

In [7]:
print(_)

 This is a test.


The instruction seems to be simple, asking for the writing of just one sentence with specific content and purpose in mind – confirming whether an environment or system works correctly by executing what appears to be straightforward commands. The simplicity lies both in its directness (a command that should yield minimal output if functional) as well as a single-step execution without additional complexity like loops, conditional statements, external resources, or advanced coding techniques commonly included at higher difficulty levels.


# Running in Docker

## Run Docker image
```cmd
docker run -it \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

## Pull Model

```cmd
docker exec -it ollama bash
ollama pull phi3
```